In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'   

import numpy as np
import torch
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import classification_report, f1_score
import torchvision.transforms as transforms

PyTorch: 2.5.1, CUDA: True


In [3]:
 
 
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.RandomHorizontalFlip(p=0.2), 
    transforms.RandomRotation(10),   
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1)),   
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

 
train_dataset = datasets.ImageFolder("C:/Users/SOHAM/python/kaggle_WCD/data/train", transform=train_transform)
val_dataset = datasets.ImageFolder("C:/Users/SOHAM/python/kaggle_WCD/data/valid", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)  

In [4]:
num_classes = len(train_dataset.classes)
num_classes

15

In [5]:
class WheatCNN(nn.Module):
    def __init__(self, num_classes=15):  
        super(WheatCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = self.global_avg_pool(x)
        x = x.view(-1, 256)
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

 

In [6]:


 
 
 
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WheatCNN(num_classes=15).to(device)
print(f"Model on {device}")

optimizer = optim.Adam(model.parameters(), lr=3e-3, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
 
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        if batch_idx % 10 == 0:
            print(f'Batch {batch_idx}/{len(loader)} Loss: {loss.item():.3f}')
    scheduler.step()
    return running_loss/len(loader), correct/total

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_acc = correct / total
    val_f1 = f1_score(all_labels, all_preds, average='weighted')
    return running_loss/len(loader), val_acc, val_f1, all_preds, all_labels

 
epochs = 30
best_val_acc = 0
best_val_f1 = 0
patience_counter = 0
patience = 10

 
print("="*80)

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/30")
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_f1, preds, labels = validate_epoch(model, val_loader, criterion, device)
    
    print(f"  Train: {train_acc:.1%} (Loss: {train_loss:.3f})")
    print(f"  Val:   {val_acc:.1%} (F1: {val_f1:.3f})")
    print("-" * 50)
    
    if val_acc > best_val_acc or val_f1 > best_val_f1:
        best_val_acc = max(best_val_acc, val_acc)
        best_val_f1 = max(best_val_f1, val_f1)
        torch.save(model.state_dict(), 'best_wheat_cnn_aggressive.pth')
        patience_counter = 0
        print(f" NEW BEST: {best_val_acc:.1%} (F1: {best_val_f1:.3f})")
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

print("\n FINAL REPORT")
print(classification_report(labels, preds, target_names=train_dataset.classes))
print(f"BEST: best_wheat_cnn_aggressiv.pth | {best_val_acc:.1%} F1 {best_val_f1:.3f}")



Model on cuda

Epoch 1/30
Batch 0/410 Loss: 2.732
Batch 10/410 Loss: 2.653


KeyboardInterrupt: 

In [29]:
train_dataset = datasets.ImageFolder("C:/Users/SOHAM/python/kaggle_WCD/data/train", transform=train_transform)
val_dataset = datasets.ImageFolder("C:/Users/SOHAM/python/kaggle_WCD/data/valid", transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)



print(f" Train: {len(train_dataset)} | Val: {len(val_dataset)}")  

 
checkpoint = torch.load(r"C:\Users\SOHAM\best_wheat_cnn_aggressive.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print("1")


 Train: 13104 | Val: 600


In [7]:
model.load_state_dict(torch.load('best_wheat_cnn_aggressive.pth', map_location=device, weights_only=False))
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)  # 
 

 
for epoch in range(10, 30):   
    print(f"\nEpoch {epoch+1}/30")
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_f1, preds, labels = validate_epoch(model, val_loader, criterion, device)
    
    print(f"  Train: {train_acc:.1%} (Loss: {train_loss:.3f})")
    print(f"  Val:   {val_acc:.1%} (F1: {val_f1:.3f})")
    
     
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_wheat_cnn_aggressive.pth')
        print(f" NEW BEST: {best_val_acc:.1%}")


Epoch 11/30
Batch 0/410 Loss: 1.274
Batch 10/410 Loss: 1.402
Batch 20/410 Loss: 1.784
Batch 30/410 Loss: 1.740
Batch 40/410 Loss: 1.731
Batch 50/410 Loss: 1.468
Batch 60/410 Loss: 1.852
Batch 70/410 Loss: 1.370
Batch 80/410 Loss: 1.468
Batch 90/410 Loss: 1.403
Batch 100/410 Loss: 1.661
Batch 110/410 Loss: 1.077
Batch 120/410 Loss: 1.495
Batch 130/410 Loss: 1.226
Batch 140/410 Loss: 1.313
Batch 150/410 Loss: 1.528
Batch 160/410 Loss: 1.388
Batch 170/410 Loss: 1.806
Batch 180/410 Loss: 1.214
Batch 190/410 Loss: 1.624
Batch 200/410 Loss: 1.398
Batch 210/410 Loss: 1.708
Batch 220/410 Loss: 1.724
Batch 230/410 Loss: 1.341
Batch 240/410 Loss: 1.418
Batch 250/410 Loss: 1.284
Batch 260/410 Loss: 1.475
Batch 270/410 Loss: 1.456
Batch 280/410 Loss: 1.329
Batch 290/410 Loss: 1.100
Batch 300/410 Loss: 1.931
Batch 310/410 Loss: 1.513
Batch 320/410 Loss: 1.728
Batch 330/410 Loss: 1.410
Batch 340/410 Loss: 1.177
Batch 350/410 Loss: 1.243
Batch 360/410 Loss: 1.774
Batch 370/410 Loss: 1.304
Batch 380/

KeyboardInterrupt: 

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
 
train_dir = r"C:/Users/SOHAM/python/kaggle_WCD/data/train"
val_dir   = r"C:/Users/SOHAM/python/kaggle_WCD/data/valid"
 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)),
])

 
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(val_dir,   transform=val_transform)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes, "num_classes:", num_classes)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True)

Device: cuda
Classes: ['Aphid', 'Black Rust', 'Blast', 'Brown Rust', 'Common Root Rot', 'Fusarium Head Blight', 'Healthy', 'Leaf Blight', 'Mildew', 'Mite', 'Septoria', 'Smut', 'Stem fly', 'Tan spot', 'Yellow Rust'] num_classes: 15


In [3]:
weights = models.ResNet18_Weights.IMAGENET1K_V1  # pretrained ImageNet
model = models.resnet18(weights=weights)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

ckpt_path = "best_resnet18_wheat.pth"

In [4]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        preds = out.argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    return running_loss / total, correct / total


def eval_epoch(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)

            running_loss += loss.item() * xb.size(0)
            preds = out.argmax(1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

    return running_loss / total, correct / total
 
for name, param in model.named_parameters():
    if not name.startswith("fc."):
        param.requires_grad = False

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                       lr=1e-3, weight_decay=1e-4)

best_val_acc = 0.0
num_epochs_head = 5

for epoch in range(num_epochs_head):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = eval_epoch(model, val_loader, criterion, device)

    print(f"[Head] Epoch {epoch+1}/{num_epochs_head} "
          f"Train Loss: {train_loss:.3f} Acc: {train_acc:.3f} "
          f"Val Loss: {val_loss:.3f} Acc: {val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "best_val_acc": best_val_acc,
        }, ckpt_path)
        print(" New BEST (head):", best_val_acc)

for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

num_epochs_ft = 20
for epoch in range(num_epochs_ft):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = eval_epoch(model, val_loader, criterion, device)
    scheduler.step()

    print(f"[FT] Epoch {epoch+1}/{num_epochs_ft} "
          f"Train Loss: {train_loss:.3f} Acc: {train_acc:.3f} "
          f"Val Loss: {val_loss:.3f} Acc: {val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "epoch": num_epochs_head + epoch,
            "model_state_dict": model.state_dict(),
            "best_val_acc": best_val_acc,
        }, ckpt_path)
        print("New BEST (fine‑tune):", best_val_acc)

print("Training done. Best val acc:", best_val_acc)

[Head] Epoch 1/5 Train Loss: 1.846 Acc: 0.503 Val Loss: 1.783 Acc: 0.559
 New BEST (head): 0.5587786259541985
[Head] Epoch 2/5 Train Loss: 1.549 Acc: 0.614 Val Loss: 1.707 Acc: 0.579
 New BEST (head): 0.5786259541984733
[Head] Epoch 3/5 Train Loss: 1.496 Acc: 0.632 Val Loss: 1.697 Acc: 0.623
 New BEST (head): 0.6229007633587786
[Head] Epoch 4/5 Train Loss: 1.480 Acc: 0.645 Val Loss: 1.662 Acc: 0.600
[Head] Epoch 5/5 Train Loss: 1.455 Acc: 0.658 Val Loss: 1.684 Acc: 0.609
[FT] Epoch 1/20 Train Loss: 1.267 Acc: 0.736 Val Loss: 1.248 Acc: 0.777
New BEST (fine‑tune): 0.7770992366412214
[FT] Epoch 2/20 Train Loss: 1.045 Acc: 0.824 Val Loss: 1.063 Acc: 0.844
New BEST (fine‑tune): 0.8442748091603054
[FT] Epoch 3/20 Train Loss: 0.945 Acc: 0.866 Val Loss: 1.017 Acc: 0.844
[FT] Epoch 4/20 Train Loss: 0.880 Acc: 0.890 Val Loss: 1.007 Acc: 0.849
New BEST (fine‑tune): 0.8488549618320611
[FT] Epoch 5/20 Train Loss: 0.835 Acc: 0.908 Val Loss: 0.945 Acc: 0.902
New BEST (fine‑tune): 0.9022900763358779


KeyboardInterrupt: 

In [33]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
test_dir = r"C:/Users/SOHAM/python/kaggle_WCD/data/test"

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)),
])

test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True)

print("Test classes:", test_dataset.classes)

# 2) Rebuild ResNet18 and load best weights
model = models.resnet18(weights=None)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 15)   # 15 classes
model.to(device)

ckpt = torch.load("best_resnet18_wheat.pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()
print("Loaded best val acc:", ckpt["best_val_acc"])

# 3) Evaluate on test
correct, total = 0, 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

test_acc = correct / total
print(f"Test accuracy: {test_acc:.4f}  ({correct}/{total})")


Test classes: ['aphid_test', 'black_rust_test', 'blast_test', 'brown_rust_test', 'common_root_rot_test', 'fusarium_head_blight_test', 'healthy_test', 'leaf_blight_test', 'mildew_test', 'mite_test', 'septoria_test', 'smut_test', 'stem_fly_test', 'tan_spot_test', 'yellow_rust_test']


C:\Users\SOHAM\AppData\Local\Temp\ipykernel_8028\3122050831.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("best_resnet18_wheat.pth", map_location=de

Loaded best val acc: 0.916030534351145
Test accuracy: 0.8800  (660/750)


In [32]:
import torch
from PIL import Image
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

 
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225)),
])

classes = test_dataset.classes 
 
def predict_image(img_path, model, transform, classes, device):
    model.eval()
    image = Image.open(img_path).convert("RGB")
    x = transform(image)          # C,H,W
    x = x.unsqueeze(0).to(device) # 1,C,H,W

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        pred_idx = probs.argmax(1).item()
        pred_prob = probs[0, pred_idx].item()

    return classes[pred_idx], pred_prob

# ----- 3) USE IT -----
img_path = r"C:\Users\SOHAM\OneDrive\画像\Camera Roll\WIN_20251116_09_23_29_Pro.jpg" # change to your image path
label, conf = predict_image(img_path, model, img_transform, classes, device)
 
label = label.replace("_test", "").replace("_", " ").title()
if conf * 100 > 75:
    print(f"Predicted: {label}  (confidence: {conf:.3f})")
else:
    print("Not wheat")
 


Not wheat


In [10]:
classes

['aphid_test',
 'black_rust_test',
 'blast_test',
 'brown_rust_test',
 'common_root_rot_test',
 'fusarium_head_blight_test',
 'healthy_test',
 'leaf_blight_test',
 'mildew_test',
 'mite_test',
 'septoria_test',
 'smut_test',
 'stem_fly_test',
 'tan_spot_test',
 'yellow_rust_test']

In [35]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

model.eval()
all_labels = []
all_preds  = []

with torch.no_grad():
    for xb, yb in test_loader:       
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(1)

        all_labels.extend(yb.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds  = np.array(all_preds)

cm = confusion_matrix(all_labels, all_preds)   
print("Confusion matrix:\n", cm)


Confusion matrix:
 [[49  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0 46  0  3  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0 50  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 44  0  0  0  0  2  0  3  0  0  0  1]
 [ 0  0  0  0 47  0  0  0  0  2  0  1  0  0  0]
 [ 0  0  0  0  0 49  0  0  0  0  0  1  0  0  0]
 [ 1  0  2  0  0  0  4  0  0  0  0  0  0  0 43]
 [ 2  1  0  0  0  0  0 42  1  0  1  0  0  3  0]
 [ 0  0  0  0  0  0  0  0 49  1  0  0  0  0  0]
 [ 4  1  0  0  0  0  0  4  0 40  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  2  0  0 48  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 50  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 50  0  0]
 [ 1  0  0  0  1  0  0  4  0  1  1  0  0 42  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 50]]


In [36]:
# https://drive.google.com/drive/folders/1Ymm1ILnmFEChThGNbQTd3KTe-eeZwtZn?usp=drive_link

In [37]:
# What ive done is, tried implementing a new CNN model but due to overfitting, I used a pre trained model. If the accuracy to predict is less than 80%, 
# output as no wheat in frame as the pre trained model is accurate when it comes to wheat diseases.

In [38]:
# Create a OpenCV pipeline implementing the above idea.